In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


sns.set()

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
# from IPython.core.display import display, HTML
from IPython.display import display, HTML

display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# import pandas_profiling

# ----------------- 학습 ----------------------
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# ----------------- 모델 ----------------------
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

# ----------------- 평가 ----------------------
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
# from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve

# from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve

# 데이터 로드

In [3]:
train = pd.read_csv('../data/traindf.csv')
test = pd.read_csv('../data/testdf.csv')
train = train.rename(columns={'Unnamed: 0':'날짜'})
test = test.rename(columns={'Unnamed: 0':'날짜'})

# train.info(),test.info()

# optuna


<pre>
튜닝 횟수 : 50
-------------------
Best param: {'reg_alpha': 7.860977201898474}
Best param: {'reg_alpha': 7.860977201898474}
-------------------
Best score: 0.8714655019174178

In [4]:
X_train = train.drop(['날짜','등락률','등락여부'],axis=1)
y_train = train['등락률']
X_test = test.drop(['날짜','등락률','등락여부'],axis=1)
y_test= test['등락률']

In [5]:
import optuna

class sklearn.linear_model.Ridge(alpha=1.0, *, fit_intercept=True, copy_X=True, max_iter=None, tol=0.0001, solver='auto', positive=False, random_state=None)


In [6]:
def my_objective_ri(trial, data=X_train, target=y_train):
    
    #train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param = {
        # 'metric': 'rmse', 
        'alpha': 1,
        'max_iter': 100,  #100
        'tol' : trial.suggest_float('tol', 1e-2, 10.0),
        # 'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10.0),  #reg_alpha=0.0
        # 'learning_rate': trial.suggest_loguniform('learning_rate', 0.01,0.1),
        # 'max_depth': trial.suggest_categorical('max_depth', [1,30]),
        #'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        #'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        #'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        #'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        #'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = Ridge(**param)  
    
    model.fit(X_train,y_train)
              # ,eval_set=[(X_test,y_test)]
              # ,early_stopping_rounds=10,verbose=False)
    pred = model.predict(X_test)
    
    rmse = mean_squared_error(y_test, pred,squared=False)
    return rmse

In [11]:

study = optuna.create_study(direction='minimize')


# y_train = np.expm1(y_train_log)
# y_test  = np.expm1(y_test_log)


# my_objective_LGBM(trial):
# study.optimize(my_objective_LGBM, n_trials=50)

# def my_objective_LGBM(trial, data=X_train, target=y_train):
study.optimize(lambda trial : my_objective_ri(trial, y_train,y_test), n_trials=50)

[I 2023-06-19 11:26:58,585] A new study created in memory with name: no-name-aa3ed49f-9e58-49dc-8dce-4824c1054cf3
[I 2023-06-19 11:26:58,587] Trial 0 finished with value: 0.8714655019174178 and parameters: {'reg_alpha': 7.727825155835548}. Best is trial 0 with value: 0.8714655019174178.
[I 2023-06-19 11:26:58,599] Trial 1 finished with value: 0.8714655019174178 and parameters: {'reg_alpha': 5.049168601638939}. Best is trial 0 with value: 0.8714655019174178.
[I 2023-06-19 11:26:58,602] Trial 2 finished with value: 0.8714655019174178 and parameters: {'reg_alpha': 8.00547322049386}. Best is trial 0 with value: 0.8714655019174178.
[I 2023-06-19 11:26:58,611] Trial 3 finished with value: 0.8714655019174178 and parameters: {'reg_alpha': 5.281214856835448}. Best is trial 0 with value: 0.8714655019174178.
[I 2023-06-19 11:26:58,618] Trial 4 finished with value: 0.8714655019174178 and parameters: {'reg_alpha': 2.941933464120425}. Best is trial 0 with value: 0.8714655019174178.
[I 2023-06-19 11:

In [12]:
print('튜닝 횟수 :', len(study.trials))
print("-------------------")
print('Best param:', study.best_trial.params)
print('Best param:', study.best_params)
print("-------------------")
print('Best score:', study.best_trial.value)
print("-------------------")
print('study.trials 전체 정보 :', study.trials)


튜닝 횟수 : 50
-------------------
Best param: {'reg_alpha': 7.727825155835548}
Best param: {'reg_alpha': 7.727825155835548}
-------------------
Best score: 0.8714655019174178
-------------------
study.trials 전체 정보 : [FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.8714655019174178], datetime_start=datetime.datetime(2023, 6, 19, 11, 26, 58, 587481), datetime_complete=datetime.datetime(2023, 6, 19, 11, 26, 58, 587481), params={'reg_alpha': 7.727825155835548}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'reg_alpha': FloatDistribution(high=10.0, log=False, low=0.01, step=None)}, trial_id=0, value=None), FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.8714655019174178], datetime_start=datetime.datetime(2023, 6, 19, 11, 26, 58, 594958), datetime_complete=datetime.datetime(2023, 6, 19, 11, 26, 58, 599945), params={'reg_alpha': 5.049168601638939}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'reg_alpha': FloatDistribution(

# 스케일링 X
* RMSE : 0.8715

In [13]:
ri=Ridge(alpha = 1,tol= 7.860977201898474)
ri.fit(X_train,y_train)
# print( xgb.coef_ )

pred = ri.predict(X_test)
rmse = mean_squared_error(y_test, pred, squared=False)
print(f"RMSE : {rmse:.4f}")

RMSE : 0.8715
